In [16]:
import pandas as pd
import sqlalchemy as sqla
import datetime as dt

In [17]:
#disk_engine = sqla.create_engine(R'sqlite:///Q:\reports_311\311_8M.db')

In [18]:
# Conversion code taken from a Plotly tutorial and slightly modified# start = dt.datetime.now()
# chunksize = 20000
# j = 0
# index_start = 1
#
# for df in pd.read_csv(R"Q:\reports_311\311_Service_Requests_Septmber.csv", chunksize=chunksize, iterator=True, encoding='utf-8',low_memory=False):
#
#     df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns
#
#     df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
#     df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])
#
#     df.index += index_start
#
#     # Remove the un-interesting columns
#
#     j+=1
#     print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))
#
#     df.to_sql('data', disk_engine, if_exists='append')
#     index_start = df.index[-1] + 1

In [19]:
disk_engine = sqla.create_engine(R'sqlite:///Q:\reports_311\311_8M.db')
df = pd.read_sql_query('SELECT * FROM data', disk_engine)
df

,index,UniqueKey,CreatedDate,ClosedDate,Agency,AgencyName,ComplaintType,Descriptor,LocationType,IncidentZip,...,VehicleType,TaxiCompanyBorough,TaxiPickUpLocation,BridgeHighwayName,BridgeHighwayDirection,RoadRamp,BridgeHighwaySegment,Latitude,Longitude,Location
0,1,51710678,2021-09-01 00:00:00.000000,2021-10-19 12:58:00.000000,DOT,Department of Transportation,Street Light Condition,Street Light Out,None,11104.0,...,None,None,None,None,None,None,None,40.744791,-73.917422,"(40.744791406467726, -73.91742167156947)"
1,2,51700093,2021-09-01 00:00:05.000000,2021-09-01 01:35:19.000000,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,10458.0,...,None,None,None,None,None,None,None,40.865358,-73.892495,"(40.86535764249122, -73.892495473103)"
2,3,51698315,2021-09-01 00:00:50.000000,2021-09-01 03:12:53.000000,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,10472.0,...,None,None,None,None,None,None,None,40.830181,-73.875717,"(40.83018108184639, -73.87571695998837)"
3,4,51698397,2021-09-01 00:00:50.000000,2021-09-01 00:52:38.000000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,10467.0,...,None,None,None,None,None,None,None,40.884150,-73.877663,"(40.88414951808322, -73.87766297790301)"
4,5,51709619,2021-09-01 00:01:00.000000,2021-09-01 12:00:00.000000,DSNY,Department of Sanitation,Dirty Conditions,E3 Dirty Sidewalk,Sidewalk,11223.0,...,None,None,None,None,None,None,None,40.597335,-73.969943,"(40.59733480704516, -73.9699432302203)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254116,1814117,52028575,2021-09-29 23:58:40.000000,2021-09-30 00:56:57.000000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10305.0,...,None,None,None,None,None,None,None,40.590757,-74.090003,"(40.59075714410544, -74.09000301726373)"
254117,1814118,52026012,2021-09-29 23:58:53.000000,2021-09-30 01:33:33.000000,NYPD,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10452.0,...,None,None,None,None,None,None,None,40.831447,-73.920162,"(40.831447302010204, -73.92016162115374)"
254118,1814119,52034506,2021-09-29 23:58:59.000000,2021-09-30 03:59:25.000000,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11237.0,...,None,None,None,None,None,None,None,40.699733,-73.913193,"(40.69973306881019, -73.91319303344328)"
254119,1814120,52028683,2021-09-29 23:59:20.000000,2021-09-30 02:08:38.000000,NYPD,New York City Police Department,Noise - Vehicle,Car/Truck Music,Street/Sidewalk,10472.0,...,None,None,None,None,None,None,None,40.827755,-73.854558,"(40.82775471005953, -73.85455792161561)"


In [20]:
q1 = pd.read_sql_query('''
SELECT AgencyName, COUNT(DISTINCT(ComplaintType)) FROM data
GROUP BY AgencyName


''', disk_engine)

In [21]:
q1

,AgencyName,COUNT(DISTINCT(ComplaintType))
0,Department for the Aging,1
1,Department of Buildings,15
2,Department of Consumer Affairs,1
3,Department of Education,1
4,Department of Environmental Protection,11
5,Department of Health and Mental Hygiene,32
6,Department of Homeless Services,2
7,Department of Housing Preservation and Develop...,13
8,Department of Information Technology and Telec...,2
9,Department of Parks and Recreation,11


In [22]:
q2 =  pd.read_sql_query('''
SELECT AgencyName, COUNT(ComplaintType) FROM data
WHERE VehicleType NOT LIKE 'None'
GROUP BY AgencyName


''', disk_engine)


In [23]:
q2

,AgencyName,COUNT(ComplaintType)
0,Taxi and Limousine Commission,59


In [24]:
q3 =  pd.read_sql_query('''
SELECT COUNT(ComplaintType),
    CASE
    WHEN AgencyName = 'Department of Transportation' THEN 'DOT'
    WHEN AgencyName = 'Taxi and Limousine Commission' THEN 'T&LC'
    ELSE AgencyName
    END AS SomeShortAgencyName
FROM data

GROUP BY SomeShortAgencyName
ORDER BY length(SomeShortAgencyName)


''', disk_engine)

In [25]:
q3

,COUNT(ComplaintType),SomeShortAgencyName
0,18921,DOT
1,1908,T&LC
2,8466,Department of Buildings
3,144,Department of Education
4,9,Department for the Aging
5,20638,Department of Sanitation
6,1805,Department of Consumer Affairs
7,5189,Department of Homeless Services
8,130351,New York City Police Department
9,1865,Economic Development Corporation
